# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

### Install

In [ ]:
!pip install -U --pre tensorflow-gpu=="2.*"
!pip install tf_slim

  Using cached https://files.pythonhosted.org/packages/b2/52/058b3372ace9d2184c1627133da6faf1964290a921a990dcd29b05e7075a/tensorflow_gpu-2.3.0rc1-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/39/96/1bf989e9b36b331cc113dd393d0fa0b952f254aca2fbf8363ce28ee1cc03/tensorflow_estimator-2.3.0rc0-py2.py3-none-any.whl
     |████████████████████████████████| 3.0MB 3.2MB/s 
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.3.0rc0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-gpu 1.14.0
    Uninstalling tensorflow-gpu-1.14.0:
      Successfully uninstalled tensorflow-gpu-1.14.

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Num GPUs Available:  1


Make sure you have `pycocotools` installed

In [ ]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1289819 sha256=606c68372c40d35cdd7ec5d9e4ce0781d34c4ec213515f23a714978d5a3a6450
  Stored in directory: /tmp/pip-ephem-wheel-cache-5jplloh2/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
  Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


### Imports

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Übungvorbereitung

### Globale Variablen

In [ ]:
# Urls
DATEI_NAME_FullIJCNN2013="FullIJCNN2013.zip"
URL_DATEI_FullIJCNN2013='https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip'

DATEI_NAME_TrainIJCNN2013="TrainIJCNN2013.zip"
URL_DATEI_TrainIJCNN2013='https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/TrainIJCNN2013.zip'

DATEI_NAME_TestIJCNN2013="TestIJCNN2013.zip"
URL_DATEI_TestIJCNN2013='https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/TestIJCNN2013.zip'

DATEI_NAME_GT_TEXT="gt.txt"
URL_DATEI_GT_TEXT='https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/gt.txt'

# Paths
DIR_Datasets='./datasets'

DIR_NAME_Full='FullIJCNN2013'

DIR_NAME_Train='TrainIJCNN2013'

DIR_NAME_test='TestIJCNN2013Download'

ReadME_DATEI='ReadMe.txt'

PATH_GT_TRAIN=str(pathlib.Path(DIR_Datasets)/DIR_NAME_Train/'gt.txt')

PATH_GT_FULL=str(pathlib.Path(DIR_Datasets)/DIR_NAME_Full/'gt.txt')

###  Datensatz herunterladen

#### FullIJCNN2013.zip

In [ ]:
PATH_TO_DOWNLOADED_full= tf.keras.utils.get_file(
    fname=DATEI_NAME_FullIJCNN2013,
    origin=URL_DATEI_FullIJCNN2013,
    cache_dir='.',
    extract=True)


1661870080/1661866983 [==============================] - 70s 0us/step


#### TrainIJCNN2013.zip

In [ ]:
PATH_TO_DOWNLOADED_train = tf.keras.utils.get_file(
    fname=DATEI_NAME_TrainIJCNN2013,
    origin=URL_DATEI_TrainIJCNN2013,
    cache_dir='.',
    extract=True)

1104183296/1104177081 [==============================] - 49s 0us/step


#### TestIJCNN2013.zip

In [ ]:
PATH_TO_DOWNLOADED_test = tf.keras.utils.get_file(
    fname=DATEI_NAME_TestIJCNN2013,
    origin=URL_DATEI_TestIJCNN2013,
    cache_dir='.',
    extract=True)

555679744/555674823 [==============================] - 26s 0us/step


#### gt.txt

In [ ]:
PATH_TO_DOWNLOADED_gt_text = tf.keras.utils.get_file(
    fname=DATEI_NAME_GT_TEXT,
    origin=URL_DATEI_GT_TEXT,
    cache_dir='.')


32768/24794 [=======================================] - 0s 4us/step


### Bildformat Konvertieren

### Reconstruct File-System


In [ ]:
##### del zip-files
os.remove(PATH_TO_DOWNLOADED_full)
os.remove(PATH_TO_DOWNLOADED_train)
os.remove(PATH_TO_DOWNLOADED_test)

In [ ]:
##### del sub-directorys
import shutil


## delete sub-directory of 'FullIJCNN2013'
p=pathlib.Path(DIR_Datasets)/DIR_NAME_Full
dir_list1=[x for x in p.iterdir() if x.is_dir()]
for sub_dir in dir_list1:
  shutil.rmtree(sub_dir)

## delete sub-directory of 'TrainIJCNN2013'
p=pathlib.Path(DIR_Datasets)/DIR_NAME_Train
dir_list1=[x for x in p.iterdir() if x.is_dir()]
for sub_dir in dir_list1:
  shutil.rmtree(sub_dir)

## delete sub-directory of 'TrainIJCNN2013Download'
p=pathlib.Path(DIR_Datasets)/DIR_NAME_test
dir_list1=[x for x in p.iterdir() if x.is_dir()]
for sub_dir in dir_list1:
  shutil.rmtree(sub_dir)


### PPM to JPEG

In [ ]:
def change_image_format_batch(src_path, tar_path, fmt_in, fmt_out ):
  '''
  Bildkonvertierung für alle Bilder im Stammordner von Ausgangformat in Zielforamt
  Args:
    src_path : 'string',  Stammordner,aus dem wir Bilder einlesen 
    tar_path : 'string', Stammordner, wo wir Ergebnis ablegen
    fmt_in  : 'string',  originale Bildformat
    fmt_out : 'string',  erwartete Bildformat
  '''
  import glob,os
  from PIL import Image

  if os.path.exists(src_path)==False:
    raise FileNotFoundError( 'No such file or directory:'+ src_path)
  
  img_dict=dict()
  directorys=[ subpath for subpath in os.listdir(src_path) if   os.path.isdir( os.path.join(src_path,subpath) )   ]
  #################################################
  if len(directorys)==0:
    imgPaths=glob.glob(os.path.join(src_path,'*.'+ fmt_in))
    if os.path.exists(tar_path)==False:
      os.makedirs(tar_path)
    for imgPath in imgPaths:
      im=Image.open(imgPath)
      _,imgNamePPM=os.path.split(imgPath)
      imgName,PPM=os.path.splitext(imgNamePPM)
      im.save(os.path.join(tar_path,imgName+'.'+ fmt_out))
    return


  #################################################
  for subdir in directorys:
    img_dict[subdir]=glob.glob(os.path.join(src_path,subdir,'*.'+ fmt_in))
  
  if os.path.exists(tar_path)==False:
    os.makedirs(tar_path)
  # erstelle Ordner
  for subdir,imgPaths in img_dict.items():
    newLongdir=os.path.join(tar_path,subdir)
    if os.path.exists(newLongdir)==False:
      os.makedirs(newLongdir)
    for imgPath in imgPaths:
      im=Image.open(imgPath)
      _,imgNamePPM=os.path.split(imgPath)
      imgName,PPM=os.path.splitext(imgNamePPM)
      im.save(os.path.join(tar_path,subdir,imgName+'.'+ fmt_out))

In [ ]:
src_path=pathlib.Path(DIR_Datasets)/DIR_NAME_Full
tar_path=pathlib.Path(DIR_Datasets)/(DIR_NAME_Full+'_JPG')
change_image_format_batch(
    src_path=str(src_path),
    tar_path=str(tar_path),
    fmt_in='ppm',
    fmt_out='jpg'
)

### Prepare Label

In [ ]:
### read Label-Mapping-file,and convert to csv file
import re
Label_Mapping_CSV='label_mapping.csv'

PATH_README=pathlib.Path(DIR_Datasets)/DIR_NAME_Full/ReadME_DATEI
PATH_LABEL_CSV=pathlib.Path(DIR_Datasets)/DIR_NAME_Full/Label_Mapping_CSV

# Text-Inhalt in Datei ReadMe.txt
str_readme_txt=open(str(PATH_README),'r').read()
str_list=re.findall('\d+\s=\s.+',str_readme_txt,flags=re.MULTILINE)

line_list=[  row.replace('=','; ') for row in str_list]

with open(PATH_LABEL_CSV,'w',newline ='') as f:
    print('id','name',sep=';',end='\n',file=f)
    f.write('\n'.join(line_list))


In [ ]:
### writre pbtxt using csv
import pandas as pd

df_Label=pd.read_csv(
    PATH_LABEL_CSV,
    sep=';'
)

# offset

df_Label['id']=df_Label['id']+1

#PATH_PBTXT_DATEI=str(pathlib.Path('.')/'models/research/object_detection/data/gtsrb_label_map.pbtxt')
PATH_PBTXT_DATEI=str(pathlib.Path('.')/'data/gtsrb_label_map.pbtxt')

with open(PATH_PBTXT_DATEI,'w') as f:
  for id,name in zip(df_Label['id'],df_Label['name']):
    f.write('item {\n')
    f.write('  id: {}\n'.format(id))
    f.write('  name: \'{}\'\n'.format(name))
    f.write('}\n')
    f.write('\n')



In [ ]:
### extract information form gt.txt (including only training set)
import pandas as pd

df=pd.read_csv(
    PATH_GT_FULL,
    sep=';',
    names=['filename','xmin','ymin','xmax','ymax','class'])
df['filename']=df['filename'].str.replace('.ppm','.jpg')
df['class']=df['class']+1
df

,filename,xmin,ymin,xmax,ymax,class
0,00000.jpg,774,411,815,446,12
1,00001.jpg,983,388,1024,432,41
2,00001.jpg,386,494,442,552,39
3,00001.jpg,973,335,1031,390,14
4,00002.jpg,892,476,1006,592,40
...,...,...,...,...,...,...
1208,00896.jpg,808,412,846,450,16
1209,00897.jpg,804,526,828,550,43
1210,00898.jpg,323,504,355,536,5
1211,00898.jpg,817,510,849,542,5


In [ ]:
df_train=df.loc[df['filename']<='00599.jpg']
df_train

,filename,xmin,ymin,xmax,ymax,class
0,00000.jpg,774,411,815,446,12
1,00001.jpg,983,388,1024,432,41
2,00001.jpg,386,494,442,552,39
3,00001.jpg,973,335,1031,390,14
4,00002.jpg,892,476,1006,592,40
...,...,...,...,...,...,...
847,00570.jpg,881,416,914,449,10
848,00571.jpg,1287,361,1308,384,18
849,00575.jpg,403,474,435,506,39
850,00593.jpg,584,510,608,534,39


In [ ]:
df_test=df.loc[df['filename']>'00599.jpg']
df_test

,filename,xmin,ymin,xmax,ymax,class
852,00601.jpg,82,450,145,508,8
853,00602.jpg,1268,555,1299,586,9
854,00602.jpg,443,543,474,574,9
855,00603.jpg,361,445,417,500,11
856,00604.jpg,365,482,437,546,31
...,...,...,...,...,...,...
1208,00896.jpg,808,412,846,450,16
1209,00897.jpg,804,526,828,550,43
1210,00898.jpg,323,504,355,536,5
1211,00898.jpg,817,510,849,542,5


In [ ]:
df_merged_train=pd.merge(df_train, df_Label, how='left', left_on=['class'], right_on=['id'])
df_merged_train

,filename,xmin,ymin,xmax,ymax,class,id,name
0,00000.jpg,774,411,815,446,12,12,priority at next intersection (danger)
1,00001.jpg,983,388,1024,432,41,41,roundabout (mandatory)
2,00001.jpg,386,494,442,552,39,39,keep right (mandatory)
3,00001.jpg,973,335,1031,390,14,14,give way (other)
4,00002.jpg,892,476,1006,592,40,40,keep left (mandatory)
...,...,...,...,...,...,...,...,...
847,00570.jpg,881,416,914,449,10,10,no overtaking (prohibitory)
848,00571.jpg,1287,361,1308,384,18,18,no entry (other)
849,00575.jpg,403,474,435,506,39,39,keep right (mandatory)
850,00593.jpg,584,510,608,534,39,39,keep right (mandatory)


In [ ]:
df_merged_test=pd.merge(df_test, df_Label, how='left', left_on=['class'], right_on=['id'])
df_merged_test

,filename,xmin,ymin,xmax,ymax,class,id,name
0,00601.jpg,82,450,145,508,8,8,speed limit 100 (prohibitory)
1,00602.jpg,1268,555,1299,586,9,9,speed limit 120 (prohibitory)
2,00602.jpg,443,543,474,574,9,9,speed limit 120 (prohibitory)
3,00603.jpg,361,445,417,500,11,11,no overtaking (trucks) (prohibitory)
4,00604.jpg,365,482,437,546,31,31,snow (danger)
...,...,...,...,...,...,...,...,...
356,00896.jpg,808,412,846,450,16,16,no traffic both ways (prohibitory)
357,00897.jpg,804,526,828,550,43,43,restriction ends (overtaking (trucks)) (other)
358,00898.jpg,323,504,355,536,5,5,speed limit 70 (prohibitory)
359,00898.jpg,817,510,849,542,5,5,speed limit 70 (prohibitory)


### JPEG to TFRecord

In [ ]:
def create_one_tf_example_from_Img(filename,df_merged,image_dir):
  from object_detection.utils import dataset_util
  path_Image=str(pathlib.Path(image_dir)/filename) ## image path
  image_string = open(path_Image, 'rb').read()
  image_shape = tf.image.decode_jpeg(image_string).shape

  height = image_shape[0] # Image height
  width = image_shape[1] # Image width
  filename = str.encode(filename) # Filename of the image. Empty if image is not from file
  encoded_image_data = image_string # Encoded image bytes
  image_format = b'jpeg' # b'jpeg' or b'png'

  ## filename-specified sub-df
  sub_df=df_merged.loc[df_merged['filename'].isin([filename.decode()])]

  xmins = sub_df['xmin']/width # List of normalized left x coordinates in bounding box (1 per box)
  xmaxs = sub_df['xmax']/width # List of normalized right x coordinates in bounding box
             # (1 per box)
  ymins = sub_df['ymin']/height # List of normalized top y coordinates in bounding box (1 per box)
  ymaxs = sub_df['ymax']/height # List of normalized bottom y coordinates in bounding box
             # (1 per box)
  classes_text = sub_df['name'].str.encode("utf-8") # List of string class name of bounding box (1 per box)
  classes = sub_df['class'] # List of integer class id of bounding box (1 per box)





  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
  }))
  return tf_example


def create_gtsrb_tf_records(image_dir,output_dir,df_merged,record_file):
  filenames=pd.unique(df_merged['filename'])
  ###
  if os.path.exists(output_dir)== False:
    os.mkdir(output_dir)
  path_record_file=str(pathlib.Path(output_dir)/record_file)
  with tf.io.TFRecordWriter(path_record_file) as writer:
    for filename in filenames:
      tf_example=create_one_tf_example_from_Img(filename,df_merged,image_dir)
      writer.write(tf_example.SerializeToString())

  
  

In [ ]:
#####write TFRecord-file for training set
record_file_train='train_images.tfrecords'
#output_dir='models/research/object_detection/data'
output_dir='data'


image_dir=str(pathlib.Path(DIR_Datasets)/'FullIJCNN2013_JPG')

create_gtsrb_tf_records(
    image_dir=image_dir,
    output_dir=output_dir,
    df_merged=df_merged_train,
    record_file=record_file_train
)

#####
PATH_TRAIN_RECORD=str(pathlib.Path(output_dir)/record_file_train)

TypeError: ignored

In [ ]:
#####write TFRecord-file for training set
record_file_test='test_images.tfrecords'
#output_dir='models/research/object_detection/data'
output_dir='data'

image_dir=str(pathlib.Path(DIR_Datasets)/'FullIJCNN2013_JPG')

create_gtsrb_tf_records(
    image_dir=image_dir,
    output_dir=output_dir,
    df_merged=df_merged_test,
    record_file=record_file_test
)
#####
PATH_TEST_RECORD=str(pathlib.Path(output_dir)/record_file_test)

### read TFRecord file

In [ ]:
##### print first 2 examples
raw_dataset_train = tf.data.TFRecordDataset(PATH_TRAIN_RECORD)
for raw_record in raw_dataset_train.take(2):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

In [ ]:
##### print first 2 examples
raw_dataset_test = tf.data.TFRecordDataset(PATH_TEST_RECORD)
for raw_record in raw_dataset_test.take(2):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

### get model

In [ ]:
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_name='ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    cache_dir='.',
    untar=True)

366952448/366947246 [==============================] - 2s 0us/step


In [ ]:
%%bash
cd data
pwd

/content/data


In [ ]:
%%bash

pwd

/content


### Config file

In [ ]:
%%bash
cd models/research/
python object_detection/builders/model_builder_test.py

2020-07-09 01:23:13.197592: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
%%bash
export PYTHONPATH=$PYTHONPATH:/content/models

cd models/research
PIPELINE_CONFIG_PATH="/content/model/pipeline_ssd_resnet50_v1.config"
MODEL_DIR="/content/model/train"
NUM_TRAIN_STEPS=50000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1
python object_detection/model_main.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --num_train_steps=${NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
    --alsologtostderr

2020-07-09 10:16:10.015848: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
W0709 10:16:12.529564 140242792073088 model_lib.py:758] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 50000
I0709 10:16:12.529834 140242792073088 config_util.py:552] Maybe overwriting train_steps: 50000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0709 10:16:12.529987 140242792073088 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0709 10:16:12.530175 140242792073088 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0709 10:16:12.530340 140242792073088 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0709 10:16:12.530560 140242792073088 model_lib.py:774] Expected number of evaluation epochs is 1, but instead encountered

In [ ]:
%%bash
pwd

/content
